In [2]:
# paper name

# motivation (of paper)

# walkthrough
# am beispiel Lebenserwartung...

In [16]:
#############################################
# imports                                   #
#############################################

from typing import List
import heapq
from collections import defaultdict
import pandas as pd
import qcr
from collections import Counter

from qcr import load_index, save_index, load_tables, get_kc, get_c, load_query, get_table_id, hash_function, build_index


## A Sketch-based Index for Correlated Dataset Search

### Motivation

With the ever-rising amount of data available, recent researched explored queries where we want to enlarge our dataset by finding related data. This means finding the top-k tables which are joinable and correlated to our inital dataset. As these searches can be quite long they propose a more efficient and effective way of finding these tables than just naively querying over the whole data collection, which might take quite a bit. Their proposed idea has shown to achieve better results than other approaches regarding ranking accruacy and recall. 
In this notebook we want to offer you an understandable explanation of the proposed Solution and hope you are able to grasp all of it.

### Discovering Data with joinable keys and correlated data.

Steps of the algorithm:
1. build index of tables in DB
    - split larger tables into 2-col-tables (cross product)
    - creating a sketch of size n of each table in the Database
    - build the index
    
    
2. query
    - creating a sketch of the query table
    - find correlated and joinable tables for the query-table
(finding set overlap between the two sketches ?)
    

### building the index:

1. all tables with more then 2 columns are split:
    - all numerical columns are combined with all columns containig categorical values
<br>
2. all numeric values (k) per table are hashed and stored as tuple with their categorical value (=key) c_k
    - <h(k), c_k>
    - for performance reason the sketch size is applied here: only the smallest n hashed calues are kept
<br>    
3. all categorical keys (c_k) are modified according to their hashed values (h(k))
    - if the hashed value is below or above the median of all table values, c_k is categorized in -c_k or +c_k respectively
    - <h(k), +/-c_k>
    - this is used to identify corrolation
<br>
4. picking a specific sample (=sketch) of size n per table, using n tuples with smallest hash-value
    - this way the samples are comparable


#### Step 1
In this example we want to find data correlating to the life expectancy in certain countries. 
We start by building the index. Therefore we first load 3 tables conatining example data. 

In [31]:
# part 1: building the index

## The following Notebook will walk you through the steps using the following sample data. #TODO: sample data with less or more columns? Table with more columns?
## The tables are used to create our index.

t1 = pd.read_csv('data/t1.csv')
t1.columns.name = 't1'
display(t1)

t2 = pd.read_csv('data/t2.csv')
t2.columns.name = 't2'
display(t2)

t3 = pd.read_csv('data/t3.csv')
t3.columns.name = 't3'
display(t3)

t1,Country,Alcohol
0,Armenia,3.702667
1,Colombia,4.419333
2,Equatorial Guinea,7.342000
3,Germany,11.628667
4,Kazakhstan,6.641333
5,Montenegro,2.584286
6,Nicaragua,3.596667
7,Nigeria,8.646667
8,Paraguay,5.527333
9,United States of America,8.579333


t2,Country,BMI
0,Armenia,44.70625
1,Colombia,49.54375
2,Equatorial Guinea,17.85625
3,Germany,51.99375
4,Kazakhstan,45.15625
5,Montenegro,50.48750
6,Nicaragua,42.68125
7,Nigeria,19.75000
8,Paraguay,39.52500
9,United States of America,58.45000


t3,Country,Area (sq. mi.)
0,Armenia,29800.0
1,Colombia,1138910.0
2,Equatorial Guinea,28051.0
3,Germany,357021.0
4,Kazakhstan,2717300.0
5,Montenegro,NaN
6,Nicaragua,129494.0
7,Nigeria,923768.0
8,Paraguay,406750.0
9,United States of America,NaN


In [30]:
# 0. initialize index ##TODO: Wieso inverted_index leer bauen und dann table printen?

inverted_index = defaultdict(set)

## sample run  with t1: showing alcohol consuption per country

table = t1

print(table)

t1                   Country    Alcohol
0                    Armenia   3.702667
1                   Colombia   4.419333
2          Equatorial Guinea   7.342000
3                    Germany  11.628667
4                 Kazakhstan   6.641333
5                 Montenegro   2.584286
6                  Nicaragua   3.596667
7                    Nigeria   8.646667
8                   Paraguay   5.527333
9   United States of America   8.579333


In [6]:
# 1. extracting numerical column (feature) and categorical column (key)

cat_col = get_kc(table)
num_col = get_c(table)

print("cat_col:")
print(cat_col)
print("num_col:")
print(num_col)

# (if a table consists of more than 2 cols, all numerical and categorical columns are combined into 2-col-tables (cross product). here, we simply keep using our original table
table

cat_col:
['Armenia', 'Colombia', 'Equatorial Guinea', 'Germany', 'Kazakhstan', 'Montenegro', 'Nicaragua', 'Nigeria', 'Paraguay', 'United States of America']
num_col:
[3.7026666666666666, 4.419333333333333, 7.342, 11.628666666666668, 6.641333333333334, 2.584285714285714, 3.5966666666666667, 8.646666666666667, 5.527333333333333, 8.579333333333333]


t1,Country,Alcohol
0,Armenia,3.702667
1,Colombia,4.419333
2,Equatorial Guinea,7.342000
3,Germany,11.628667
4,Kazakhstan,6.641333
5,Montenegro,2.584286
6,Nicaragua,3.596667
7,Nigeria,8.646667
8,Paraguay,5.527333
9,United States of America,8.579333


#### Step 2
Now we have tables conaining only one categorical column and a numercial column. <br>
We can start hashing the numerical columns. For performance reasons it is advised to limit the sketch size. <br>
Allthough not necessary in this small case, we want to emphasize the scalability of this apporoach and used the limit of 3.

In [33]:
# 2. create sketch by hashing numerical values and picking n smallest. #TODO: oben steht numerische Spalten hashen. Hier jetzt aber beide?

table['hashed_values'] = hash_function(table['Alcohol'])
print(table)
hashed_table = table[['hashed_values', 'Country']]
#hashed_table = table[['Alcohol', 'Country']]

# now we only keep n rows with the smallest hashed values. those rows form the sketch for this table:

sketch = hashed_table.nsmallest(3, 'hashed_values')
#sketch = hashed_table.nsmallest(3, 'Alcohol')
print('\nsketch of t1:')
sketch


t1                   Country    Alcohol  hashed_values
0                    Armenia   3.702667   3.619010e-40
1                   Colombia   4.419333   3.619010e-40
2          Equatorial Guinea   7.342000   3.619010e-40
3                    Germany  11.628667   3.619010e-40
4                 Kazakhstan   6.641333   3.619010e-40
5                 Montenegro   2.584286   3.619010e-40
6                  Nicaragua   3.596667   3.619010e-40
7                    Nigeria   8.646667   3.619010e-40
8                   Paraguay   5.527333   3.619010e-40
9   United States of America   8.579333   3.619010e-40

sketch of t1:


t1,hashed_values,Country
0,3.619010e-40,Armenia
1,3.619010e-40,Colombia
2,3.619010e-40,Equatorial Guinea


#### Step 3
We now label our hashed_values according to the mean of the hashed_values.<br>
Any hashed_value below our mean has it's key (or categorical column) concatenated with "-1", while any hashed_value above the mean hast it's concatenated with "+1".

In [37]:
# 3. categorize keys by value and use as new key

# first we calculate the mean of all values of this tables numeric column
mue = sketch['hashed_values'].mean()


# then label key by > median (+key) or < median (-key)
#NOT# hash categorized term
sketch['labeled_keys'] = [f'{key}{"+1" if value > mue else "-1"}' for value, key in sketch.values]

print(sketch)

t1  hashed_values            Country         labeled_keys
0    3.619010e-40            Armenia            Armenia-1
1    3.619010e-40           Colombia           Colombia-1
2    3.619010e-40  Equatorial Guinea  Equatorial Guinea-1


#### Step 4
We finally can complete our inverted index by merging our sketches into our full inverted index. 

In [40]:
# 4. store hashed and categorized terms of sketch in inverted index #TODO: Hier bauen wir den inverted index aus den sketches? Und dannach aus den vollständigen tabellen im schnelldurchlauf?
#TODO: Reihenfolge der Zellausführungen beachten. Umbenennungen erforderlich
table_id = 't1'

for term in sketch['labeled_keys']:
    inverted_index[term].add(table_id)


inverted_index


defaultdict(set,
            {'Armenia-1': {'t1', 't3'},
             'Nicaragua-1': {'t1', 't3'},
             'Equatorial Guinea+1': {'t1'},
             'United States of America+1': {'t1', 't2'},
             'Colombia-1': {'t1', 't3'},
             'Paraguay-1': {'t1', 't2', 't3'},
             'Montenegro-1': {'t1', 't3'},
             'Kazakhstan+1': {'t1', 't2'},
             'Germany+1': {'t1', 't2'},
             'Nigeria+1': {'t1'},
             'Armenia+1': {'t2'},
             'Nicaragua+1': {'t2'},
             'Equatorial Guinea-1': {'t1', 't2', 't3'},
             'Colombia+1': {'t2'},
             'Montenegro+1': {'t2'},
             'Nigeria-1': {'t2', 't3'},
             'United States of America-1': {'t3'},
             'Kazakhstan-1': {'t3'},
             'Germany-1': {'t3'}})

In [34]:
### the above is an example of one table, the full code can be found in qcr.py

# now we also import all 3 tables into one index
build_index([t1, t2, t3])
inverted_index = load_index()
inverted_index

defaultdict(set,
            {'Armenia-1': {'t1', 't3'},
             'Nicaragua-1': {'t1', 't3'},
             'Equatorial Guinea+1': {'t1'},
             'United States of America+1': {'t1', 't2'},
             'Colombia-1': {'t1', 't3'},
             'Paraguay-1': {'t1', 't2', 't3'},
             'Montenegro-1': {'t1', 't3'},
             'Kazakhstan+1': {'t1', 't2'},
             'Germany+1': {'t1', 't2'},
             'Nigeria+1': {'t1'},
             'Armenia+1': {'t2'},
             'Nicaragua+1': {'t2'},
             'Equatorial Guinea-1': {'t2', 't3'},
             'Colombia+1': {'t2'},
             'Montenegro+1': {'t2'},
             'Nigeria-1': {'t2', 't3'},
             'United States of America-1': {'t3'},
             'Kazakhstan-1': {'t3'},
             'Germany-1': {'t3'}})

As we now have build our inverted index for the dataset to be searched we can search it for data correlating to our query table.
### Querying the index
#### Step 1
We start by building the inverted index for the sketch of our query data.

In [41]:
# query table: (key & taget)
q = pd.DataFrame([[],[]])#TODO: ich denke mal hier fehlen noch daten?
# land # lebenserwartung #
# ...


# as above:
# 1. build sketch of query table
sketch = qcr.create_sketch(q[0], q[1], hash_function(), n=5)
# 2. generate terms
terms = qcr.generate_term_keys(sketch)

# 3.  inverse values of sketch
# for negative correlation
inverse_terms = generate_term_keys(
    list(map((lambda key_value: (key_value[0], -key_value[1])), sketch)), h
    ) # same function as above, input is inverted




KeyError: 0

#### Step 2
Using our new inverted index for the query table we can search the database. 
# Welche terms auswählen? Erklären

In [19]:
# execute query
# 1. load idex
inverted_index = load_index()

# count how many tables match the sketches terms #TODO: what terms? unklar
result = Counter()
result.update(
    "+:" + table_id for term in terms for table_id in inverted_index[term]
)
result.update(
    "-:" + table_id for term in inverse_terms for table_id in inverted_index[term]
)

sketch = result.most_common(10)

NameError: name 'terms' is not defined

In [ ]:
#4. a _mirror image_ is of each sketch is build
#    - sign of value and key is inverted
#    - this is used to identify inversed correlation

In [42]:
### Query dataset (Q)

df_Q = pd.DataFrame({'movies':['A','B','C','D','E'], 
                     'budget in mil €':[100, 200, 500, 300, 300],
                     'stars':[2,3,4,4.8,3]})
display(df_Q)

print()
print('hashed df:')
h_k = df_Q['budget in mil €'].apply(lambda k: hash(k))##TODO: hash keinen effekt außer bei 4.8? -> andere hashfunktion
df_Q['budget in mil €'] = h_k

h_k = df_Q['stars'].apply(lambda k: hash(k))
df_Q['stars'] = h_k

display(df_Q)

,movies,budget in mil €,stars
0,A,100,2.0
1,B,200,3.0
2,C,500,4.0
3,D,300,4.8
4,E,300,3.0



hashed df:


,movies,budget in mil €,stars
0,A,100,2
1,B,200,3
2,C,500,4
3,D,300,1844674407370954756
4,E,300,3


In [ ]:
### example of a corrolated and joinable dataset from the corpus (c):

df_c = pd.DataFrame({'movies':['C','D','E','F','G',], 
                     'budget per staff':[1.2, 3.5, 8, 10, 4]})
display(df_c)

h_k = df_c['budget per staff'].apply(lambda k: hash(k))
df_c['budget per staff'] = h_k
display(df_c)
#df = pd.DataFrame({'movies':['C','D','E','F','G',], 'sick days':[500, 50, 150, 175, 100]})
#df

In [ ]:
# choose sketch for each table

### correlation:



In [ ]:
# find corrolated sketches and joinable sketches


### joinability:

In [ ]:
#TODO: zwei mal correltaion und joinability. Wozu?

In [48]:
#############################################
# building the index                        #
#                        overview           #
#############################################
def build_index() -> None:
    # 0. load table and initialize index
    inverted_index = defaultdict(set)
    tables = load_tables()

    # create sketch for every table and add it to the intex
    for table in tables:

        # 1. build 2-column-tables
        KC = get_kc(table)
        C = get_c(table)

        # create hash functions
        h, hu = create_hash_functions(KC, C)

        # 2. hash numerical values
        sketch = create_sketch(KC, C, h, hu)

        # 3. categorize keys by value
        terms = generate_term_keys(sketch, h)


        # 4. store terms // sketch in inverted index ???
        table_id = get_table_id(table)
        add_to_inverted_index(inverted_index, terms, table_id)


    save_index(inverted_index)

In [49]:
def find_tables(query: pd.DataFrame) -> List[str]:
    # 1. build 2-column-tables
    KC = get_kc(query)
    C = get_c(query)
    
    # create hash functions
    h, hu = create_hash_functions()  # both hash functions are used to create the sketch (hu(h(k)))
    
    # hash numerical values
    sketch = create_sketch(KC, C, h, hu)
    
    # categorize keys by value
    terms = generate_term_keys(sketch, h)
    
    # mirror image
    anti_terms = tk(
        list(map((lambda key_value: (key_value[0], -key_value[1])), sketch)), h
        )
    
    # pick smallest n terms for specific sample
    inverted_index = load_index()
    result = Counter()
    result.update(
        "+:" + table_id for term in terms for table_id in inverted_index[term]
    )
    result.update(
        "-:" + table_id for term in anti_terms for table_id in inverted_index[term]
    )
    
    sketch = result.most_common(10)
    return sketch

In [50]:
# mirror image of tuples (same function as above, mirrored input)

mirror_image = list(map((lambda key_value: (key_value[0], -key_value[1])), sketch)), h ###TODO: -keyvalue? h?

anti_terms = generate_term_keys( mirror_image )

TypeError: bad operand type for unary -: 'str'

In [ ]:
# pick smallest n terms for specific sample
    inverted_index = load_index()
    result = Counter()
    result.update(
        "+:" + table_id for term in terms for table_id in inverted_index[term]
    )
    result.update(
        "-:" + table_id for term in anti_terms for table_id in inverted_index[term]
    )
    
    sketch = result.most_common(10)
    return sketch

In [ ]:
# 2. hash numerical values

def create_sketch(
    KC: List[str],                  C: List[numeric],
    h: Callable[[str], int],        hu: Callable[[int], int],       n=100,
    ) -> List[Tuple[str, numeric]]:
    
    sketch = heapq.nsmallest(n, zip(KC, C), key=lambda x: hu(h(x[0])))
    ##       heapq.nsmallest(n, iterable  , key=funktion)
    ##       n = return n smalllest results
    ##       iterable = perform function on this data
    ##       key = use this function, pick n smallest results and put on sorted heap (heapque)
    
    return sketch

sketch = create_sketch(KC, C, h, hu)

print(sketch)

In [ ]:
# 3. categorize keys by value and use as new key

def generate_term_keys(sketch: List[Tuple[str, numeric]], h: Callable[[str], int]) -> List[int]:

    # mue = median of all values of this tables numeric column
    mue = sum([value for key, value in sketch]) / len(sketch)
    
    # categorize key by > median (+key) or < median (-key)
    # hash categorized term
    categorized_key = [h(f'{h(key)}{"+1" if value > mue else "-1"}') for key, value in sketch]
    
    return categorized_key


hashed_terms = generate_term_keys(sketch, h)

print(hashed_terms)

In [ ]:
## toy example to vizualize steps

# TODO

In [ ]:
# 4. store hashed and categorized terms  (not sketch?)  in inverted index ???

def add_to_inverted_index( inverted_index: DefaultDict[int, Set[str]],
                           hashed_terms: List[int], 
                           table_id: str                                  ) -> None:

    for term in hashed_terms:
        inverted_index[term].add(table_id)

table_id = get_table_id(table)
add_to_inverted_index(inverted_index, hashed_terms, table_id)

print(inverted_index)
# 130988216787560463481282035084846401787: {'A_0'}
# hashed categorized key/term

In [ ]:
### Query dataset (Q)

df_Q = pd.DataFrame({'movies':['A','B','C','D','E'], 
                     'budget in mil €':[100, 200, 500, 300, 300],
                     'stars':[2,3,4,4.8,3]})
display(df_Q)

print()
print('hashed df:')
h_k = df_Q['budget in mil €'].apply(lambda k: hash(k))
df_Q['budget in mil €'] = h_k

h_k = df_Q['stars'].apply(lambda k: hash(k))
df_Q['stars'] = h_k

display(df_Q)

In [ ]:
### example of a corrolated and joinable dataset from the corpus (c):

df_c = pd.DataFrame({'movies':['C','D','E','F','G',], 
                     'budget per staff':[1.2, 3.5, 8, 10, 4]})
display(df_c)

h_k = df_c['budget per staff'].apply(lambda k: hash(k))
df_c['budget per staff'] = h_k
display(df_c)
#df = pd.DataFrame({'movies':['C','D','E','F','G',], 'sick days':[500, 50, 150, 175, 100]})
#df

In [ ]:
# choose sketch for each table

### correlation:



In [ ]:
# find corrolated sketches and joinable sketches


### joinability:

In [ ]:
def find_tables(query: pd.DataFrame) -> List[str]:
    # 1. build 2-column-tables
    KC = get_kc(query)
    C = get_c(query)
    
    # create hash functions
    h, hu = create_hash_functions()  # both hash functions are used to create the sketch (hu(h(k)))
    
    # hash numerical values
    sketch = create_sketch(KC, C, h, hu)
    
    # categorize keys by value
    terms = generate_term_keys(sketch, h)
    
    # mirror image
    anti_terms = tk(
        list(map((lambda key_value: (key_value[0], -key_value[1])), sketch)), h
        )
    
    # pick smallest n terms for specific sample
    inverted_index = load_index()
    result = Counter()
    result.update(
        "+:" + table_id for term in terms for table_id in inverted_index[term]
    )
    result.update(
        "-:" + table_id for term in anti_terms for table_id in inverted_index[term]
    )
    
    sketch = result.most_common(10)
    return sketch

In [ ]:
# mirror image of tuples (same function as above, mirrored input)

mirror_image = list(map((lambda key_value: (key_value[0], -key_value[1])), sketch)), h

anti_terms = generate_term_keys( mirror_image )

In [ ]:
# pick smallest n terms for specific sample
    inverted_index = load_index()
    result = Counter()
    result.update(
        "+:" + table_id for term in terms for table_id in inverted_index[term]
    )
    result.update(
        "-:" + table_id for term in anti_terms for table_id in inverted_index[term]
    )
    
    sketch = result.most_common(10)
    return sketch

In [ ]:
import pandas as pd
df = pd.read_csv('data/Life_Expectancy_Data.csv', sep=';')
df['Country'] = df['Country'].str.strip()

In [ ]:
df = df.groupby(['Country']).mean().reset_index(level=0)
df_small = df[['Country','Life expectancy ', 'Alcohol', ' BMI ']]
df_small

In [ ]:
df_smaller = df_small.loc[df_small['Country'].isin(['Nigeria', 'Equatorial Guinea', 'Kazakhstan', 'Paraguay', 'Colombia', 'Armenia', 'Nicaragua', 'Montenegro', 'Maledives', 'Quatar', 'Germany', 'United States of America'])]
q = df_smaller[['Country', 'Life expectancy ']]
c1 = df_smaller[['Country', 'Alcohol']]
c2 = df_smaller[['Country', ' BMI ']]
df_smaller


In [ ]:
df2 = pd.read_csv('data/countries_of_the_world.csv')
display(df2)
#display(df2['Country'])
#display(df_small['Country'])
#dfx = df2[['Country']].join(df_small[['Country']], lsuffix='_who', rsuffix='_kaggle')
#dfx
#df_merge = pd.merge(df_small, df2, on='Country',  how='left')
#df_merge
display(df2.dtypes)
display(df_small.dtypes)
df_join = df_small.join(df2, on='Country',  how='left', lsuffix='_who', rsuffix='_kaggle')
df_join

In [ ]:
import pandas as pd
df = pd.read_csv('data/Life_Expectancy_Data.csv', sep=';')
df['Country'] = df['Country'].str.strip()
df = df.groupby(['Country']).mean().reset_index(level=0)



df2 = pd.read_csv('data/countries_of_the_world.csv')
df2['Country'] = df2['Country'].str.strip()
display(df2)
#display(df2['Country'])
#display(df_small['Country'])
#dfx = df2[['Country']].join(df_small[['Country']], lsuffix='_who', rsuffix='_kaggle')
#dfx
#df_merge = pd.merge(df_small, df2, on='Country',  how='left')
#df_merge

#df2 = df2.astype({'Country': 'string'})
#df_small = df_small.astype({'Country': 'string'})

#df2 = df2.set_index('Country')
#display(df2)
#df_small = df_small.set_index('Country')
#display(df_small)

#df_obj = df2.select_dtypes(['object'])
#df2[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

#df_obj = df_small.select_dtypes(['object'])
#df_small[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())



display(df2.dtypes)
display(df.dtypes)
#display(df2['Country'])
#display(df_small['Country'])
#df_join = df_small.join(df2, lsuffix='_who', rsuffix='_kaggle')
#df_join

df_join = df.join(df2.set_index('Country'), on='Country' , how='left', lsuffix='_who', rsuffix='_kaggle')

#df_join = df_small.join(df2, on='Country',  how='left', lsuffix='_who', rsuffix='_kaggle')
df_join

In [ ]:
df_small = df_join[['Country','Life expectancy ', 'Alcohol', ' BMI ']]
df_small
df_smaller = df_join.loc[df_small['Country'].isin(['Nigeria', 'Equatorial Guinea', 'Kazakhstan', 'Paraguay', 'Colombia', 'Armenia', 'Nicaragua', 'Montenegro', 'Maledives', 'Quatar', 'Germany', 'United States of America'])]
display(df_smaller)
q = df_smaller[['Country', 'Life expectancy ']]
c1 = df_smaller[['Country', 'Alcohol']]
c2 = df_smaller[['Country', ' BMI ']]
c3 = df_smaller[['Country', 'Area (sq. mi.)']]
display(q)
display(c1)
display(c2)
display(c3)


In [1]:
c2.to_csv('c2.csv', index=False)
c3.to_csv('c3.csv', index=False)
q.to_csv('q.csv', index=False)

NameError: name 'c2' is not defined

### Conclusion
As you hopefully have understood the approach, we want to emphasize the efficiency and high recall proven by their experiments. An example usage of this approach is to faster train machine learning models to predict our categorical value accodring to the numerical values provided. 